In [1]:
from peak_resolver.linear_combination_fitting import *
from peak_resolver.get_models_bases import get_models, get_basis_functions, set_bounds
from peak_resolver.process_chromeleon import process_chromeleon_file
import pickle
import pathlib
from pathlib import Path
import xarray as xr
from xarray import apply_ufunc

In [2]:
root = '/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves'
acids = ['acetic', 'propanoic', 'butyric', 'valeric', 'caproic']
models = get_models(root, acids)
acids = [a.capitalize() for a in acids]
for i in list(models):
    models[i + '_err'] = np.sqrt(np.sum(models[i].resid)**2/models[i].df_resid)

acetic
propanoic
butyric
valeric
caproic


In [3]:
basis_root = '/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/CALIBRATION CURVES/calib.nc'
basis = get_basis_functions(basis_root, conc=0.5)
basis = [i.to_numpy() for i in basis]

In [14]:
ds = xr.load_dataset('/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/2023-12-07.nc')
acids_shorthand = ['ace', 'but', 'pro','val', 'cap']
for i in acids_shorthand:
    ds = ds.assign({
        'p_' + i: xr.zeros_like(ds["ds"]),
        'c_' + i: xr.zeros_like(ds["ds"])})
    
ds = ds.assign(p_ace=xr.zeros_like(ds["ds"]))
ds = ds.assign(p_but=xr.zeros_like(ds["ds"]))
ds = ds.assign(p_pro=xr.zeros_like(ds["ds"]))
ds = ds.assign(p_val=xr.zeros_like(ds["ds"]))
ds = ds.assign(p_cap=xr.zeros_like(ds["ds"]))


ds = ds.where(ds.t > 6.5).where(ds.t < 9)
ds = ds.dropna(dim='t', how='all')


In [15]:
ds

<xarray.Dataset>
Dimensions:    (side: 2, exp_time: 5, run: 3, Acetic: 2, Propionic: 2,
                Butyric: 2, Valeric: 2, Caproic: 2, t: 761)
Coordinates:
  * t          (t) float64 6.501 6.504 6.507 6.511 ... 8.988 8.991 8.995 8.998
  * exp_time   (exp_time) float64 0.0 1.0 2.0 3.0 4.0
  * run        (run) int32 1 2 3
  * side       (side) object 'F' 'R'
  * Acetic     (Acetic) int32 0 50
  * Propionic  (Propionic) int32 0 50
  * Butyric    (Butyric) int32 0 50
  * Valeric    (Valeric) int32 0 50
  * Caproic    (Caproic) int32 0 50
Data variables: (12/14)
    index      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    s          (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    ds         (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    d2s        (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    p_ace      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    c_ace      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    ...         ...
    p_pro      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    c_pro      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    p_val      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    c_val      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    p_cap      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    c_cap      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...

In [16]:
ds.sel(side='F', exp_time=1,run=1,Acetic=50, Propionic=0, Butyric=0, Valeric=0, Caproic=0).dropna(dim='t',how='all')

<xarray.Dataset>
Dimensions:    (t: 761)
Coordinates:
  * t          (t) float64 6.501 6.504 6.507 6.511 ... 8.988 8.991 8.995 8.998
    exp_time   float64 1.0
    run        int32 1
    side       <U1 'F'
    Acetic     int32 50
    Propionic  int32 0
    Butyric    int32 0
    Valeric    int32 0
    Caproic    int32 0
Data variables: (12/14)
    index      (t) float64 1.935e+03 1.936e+03 1.937e+03 ... 2.694e+03 2.695e+03
    s          (t) float64 -0.05655 -0.05634 -0.05612 ... 0.003178 0.002974
    ds         (t) float64 0.0001923 0.00022 0.0002195 ... -0.0001065 -0.0002078
    d2s        (t) float64 2.253e-06 2.817e-05 ... 2.221e-05 -0.0001033
    p_ace      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    c_ace      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    ...         ...
    p_pro      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    c_pro      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    p_val      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    c_val      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    p_cap      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    c_cap      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0

In [17]:
models['ace'].params

array([-0.00158073,  0.49769743])

In [54]:
for side in ds.coords['side'].values:
    for run in ds.coords['run'].values:
        for exp_time in ds.coords['exp_time'].values:
            for ace in ds.coords['Acetic'].values:
                for pro in ds.coords['Propionic'].values:
                    for but in ds.coords['Butyric'].values:
                        for val in ds.coords['Valeric'].values:
                            for cap in ds.coords['Caproic'].values:
                                current_selection = dict(
                                    side=side,
                                    run=run,
                                    exp_time=exp_time,
                                    Acetic=ace,
                                    Propionic=pro,
                                    Butyric=but,
                                    Valeric=val,
                                    Caproic=cap
                                )
                                
                                to_fit = ds.sel(current_selection).dropna(dim='t', how='all')['ds']
                                
                                try:
                                    composition = (ace, but, pro, val, cap)
                                    bbounds = set_bounds(composition)
                               
                                    res = lin_combination_fitting(to_fit.values, basis, bounds = bbounds)

                                    for idx, i in enumerate(acids_shorthand):

                                        if res['r'] > 0:
                                            p = res['p'][idx]
                                            ds['p_' + i].loc[current_selection] = p
                                            # ds['p_ace'].loc[current_selection] = res['p'][0]
                                            # ds['p_but'].loc[current_selection] = res['p'][1]
                                            # ds['p_pro'].loc[current_selection] = res['p'][2]
                                            # ds['p_val'].loc[current_selection] = res['p'][3]
                                            # ds['p_cap'].loc[current_selection] = res['p'][4]
                                            if p > 0: #this if statement prevents the concentration from being the y-int of the fit in case p=0
                                                ds['c_' + i].loc[current_selection] = models[i].params[0] + models[i].params[1] * p
                                            else: 
                                                ds['c_' + i].loc[current_selection] = 0

                                        else:
                                            ds['p_' + i].loc[current_selection] = np.nan
                                            ds['c_' + i].loc[current_selection] = np.nan
                                            # ds['p_ace'].loc[current_selection] = np.nan
                                            # ds['p_but'].loc[current_selection] = np.nan
                                            # ds['p_pro'].loc[current_selection] = np.nan
                                            # ds['p_val'].loc[current_selection] = np.nan
                                            # ds['p_cap'].loc[current_selection] = np.nan
                                        
                                except ValueError as ve:
                                    pass
                



/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=meth

In [55]:
ds['c_ace'].sel(side='R', exp_time=4, run=2, Acetic=50, Propionic=0, Butyric=0, Valeric=0,Caproic=0)

<xarray.DataArray 'c_ace' (t: 761)>
array([0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
...
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694, 0.11300694, 0.11300694, 0.11300694, 0.11300694,
       0.11300694])
Coordinates:
  * t          (t) float64 6.501 6.504 6.507 6.511 ... 8.988 8.991 8.995 8.998
    exp_time   float64 4.0
    run        int32 2
    side       <U1 'R'
    Acetic     int32 50
    Propionic  int32 0
    Butyric    int32 0
    Valeric    int32 0
    Caproic    int32 0

In [58]:
wd = '/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/processed_concentrations/'
df = ds.to_dataframe()
df = df.where(df['s'] > 0).dropna()
df.to_csv(wd + 'verbose_concentrations.csv')

ds2 = ds.mean(dim='t')
ds2 = ds2.drop_vars(['index', 's', 'ds', 'd2s'])
df2 = ds2.to_dataframe()
df2 = df2.where(df2['p_ace'] >= 0).dropna()
print(df2)
df2.to_csv(wd + 'concise_concentrations.csv')

                                                                   p_ace  \
side exp_time run Acetic Propionic Butyric Valeric Caproic                 
F    0.0      1   0      0         0       50      0        0.000000e+00   
                  50     0         0       0       0        5.274906e-01   
                                           50      0        5.278518e-01   
                                   50      50      0        2.725693e-01   
                         50        50      50      50       3.456109e-01   
...                                                                  ...   
R    4.0      2   50     0         0       0       0        2.302356e-01   
                                           50      0        1.258059e-01   
                                   50      50      0        1.657593e-07   
                         50        50      50      50       0.000000e+00   
              3   0      0         0       50      0        0.000000e+00   

           

In [39]:
ds2

<xarray.Dataset>
Dimensions:    (side: 2, exp_time: 5, run: 3, Acetic: 2, Propionic: 2,
                Butyric: 2, Valeric: 2, Caproic: 2)
Coordinates:
  * exp_time   (exp_time) float64 0.0 1.0 2.0 3.0 4.0
  * run        (run) int32 1 2 3
  * side       (side) object 'F' 'R'
  * Acetic     (Acetic) int32 0 50
  * Propionic  (Propionic) int32 0 50
  * Butyric    (Butyric) int32 0 50
  * Valeric    (Valeric) int32 0 50
  * Caproic    (Caproic) int32 0 50
Data variables:
    p_ace      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic) float64 ...
    c_ace      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic) float64 ...
    p_but      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic) float64 ...
    c_but      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic) float64 ...
    p_pro      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic) float64 ...
    c_pro      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic) float64 ...
    p_val      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic) float64 ...
    c_val      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic) float64 ...
    p_cap      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic) float64 ...
    c_cap      (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic) float64 ...

In [59]:
models['ace'].summary()

/Users/ianbillinge/micromamba/envs/peak_resolver/lib/python3.12/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                     5535.
Date:                Tue, 02 Jan 2024   Prob (F-statistic):           1.96e-07
Time:                        15:26:05   Log-Likelihood:                 24.211
No. Observations:                   6   AIC:                            -44.42
Df Residuals:                       4   BIC:                            -44.84
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0016      0.004     -0.386      0.719      -0.013       0.010
x1             0.4977      0.007     74.395      0.000       0.479       0.516
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   3.025
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.922
Skew:                          -0.957   Prob(JB):                        0.631
Kurtosis:                       2.834   Cond. No.                         4.05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""